<a href="https://colab.research.google.com/github/carahmel/its-feature-counter-binder-colab/blob/main/ITS_feature_counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*If prompted, select _ipykernel_ as kernel.*

### **To initialise `source_json` variable and execute script to count charging stations :**


1.   Download source file (from *https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/ch.bfe.ladestellen-elektromobilitaet_fr.json* or an alternate source) to your computer.
2.   Select the next cell and run it by clicking on the *play* button (or with the *Ctrl+Enter* keys) :

In [ ]:
## Select and run this cell ##

from google.colab import files
uploaded = files.upload()



3.   Click on *Browse..* and upload the source file that you have downloaded in Step 1. This can take a few minutes to complete.
4.   Execute the main script below by clicking on the *play* button (or with the *Ctrl+Enter* keys) :


*You can re-run the cell to get the latest results without re-uploading a source file (Steps 1-3) as long as the current Colab session remains open.*







---

### **Do not edit the main script below !**

---


In [ ]:
# @title
### !! Do not edit this cell !! ###

# import libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import branca
import datetime

# time stamp
time = datetime.datetime.now()
now = time.strftime("%Y-%m-%d %A %H:%M:%S")

# load data
lausanne = gpd.read_file("https://raw.githubusercontent.com/carahmel/its-feature-counter-binder/refs/heads/main/resources/limites_lausanne_2056.geojson")

for filename in uploaded.keys():
  source_json = filename
stations = gpd.read_file(f"/content/{source_json}").to_crs(lausanne.crs) # ensure that both datasets share the same CRS

# spatial join to identify charging stations within lausanne
stationsLS = gpd.sjoin(stations, lausanne, how='inner')

# count number of charging points per status, per station
points_available = pd.Series(stationsLS['description'].str.count("<table class=\"evse-overview status-available\">"), name = "available")
points_occupied = pd.Series(stationsLS['description'].str.count("<table class=\"evse-overview status-occupied\">"), name = "occupied")
points_reserved = pd.Series(stationsLS['description'].str.count("<table class=\"evse-overview status-reserved\">"), name = "reserved")
points_outofservice = pd.Series(stationsLS['description'].str.count("<table class=\"evse-overview status-outofservice\">"), name = "outofservice")
points_unknown = pd.Series(stationsLS['description'].str.count("<table class=\"evse-overview status-unknown\">"), name = "unknown")
pointsLS = pd.concat([points_available, points_occupied, points_reserved, points_outofservice, points_unknown], axis=1)

# create resulting dataframe
results = pd.merge(stationsLS, pointsLS, left_index=True, right_index=True)

# initialise map
fig = branca.element.Figure(height=600)
m = folium.Map([46.519630, 6.632130], tiles="cartodbpositron", zoom_start=12)
fig.add_child(m)

# create groups for layer control
available = folium.FeatureGroup(name="Available", show=True).add_to(m)
unavailable = folium.FeatureGroup(name="Unavailable/Unknown", show=True).add_to(m)  # reserved, occupied, out of service, unknown

# define function for marker colours
def colours(feature):
    if feature['properties']["Availability"] == "Available":
        return "green"
    if feature['properties']["Availability"] == "Reserved":
        return "orange"
    if feature['properties']["Availability"] == "Occupied":
        return "red"
    if feature['properties']["Availability"] == "OutOfService":
        return "black"
    if feature['properties']["Availability"] == "Unknown":
        return "gray"
    else:
        return "gray"

# add objects to map
folium.GeoJson(results[results['Availability'] == "Available"],
               name = "stations de recharge",
               marker=folium.Marker(icon=folium.Icon(icon='glyphicon-flash')),
               tooltip=folium.GeoJsonTooltip(fields=["available", "occupied", "reserved", "outofservice", "unknown"], aliases=["Available:", "Occupied:", "Reserved:", "Out of Service:", "Unknown:"]),
               style_function=lambda feature: {
                   'markerColor': colours(feature)}).add_to(available)
folium.GeoJson(results[results['Availability'] != "Available"],
               name = "stations de recharge",
               marker=folium.Marker(icon=folium.Icon(icon='glyphicon-flash')),
               tooltip=folium.GeoJsonTooltip(fields=["available", "occupied", "reserved", "outofservice", "unknown"], aliases=["Available:", "Occupied:", "Reserved:", "Out of Service:", "Unknown:"]),
               style_function=lambda feature: {
                   'markerColor': colours(feature)}).add_to(unavailable)
folium.GeoJson(lausanne,
               name = "Ville de Lausanne",
               style_function=lambda feature: {
                   "fillColor": "darkgray",
                   "fillOpacity": 0.5,
                   "color": "darkgray",
                   "weight": 2},
               show = True).add_to(m)
folium.TileLayer("OpenStreetMap", show = False).add_to(m)
folium.LayerControl().add_to(m)

# print results and map to cell
def print_results(map):
  print("STATUS OF CHARGING POINTS:")
  print("# Available:", np.sum(results['available']))
  print("# Reserved:", np.sum(results['reserved']))
  print("# Occupied:", np.sum(results['occupied']))
  print("# Out of service:", np.sum(results['outofservice']))
  print("# Unknown:", np.sum(results['unknown']))
  print(f"\nTOTAL: {np.sum([results['available'], results['reserved'], results['occupied'], results['outofservice'], results['unknown']])} charging points across {len(results)} stations")
  print(f"({now} UTC)")
  return map

print_results(m)